# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [169]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config1 import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [170]:
weather_string = '../WeatherPy/output_data/cities.csv'
weather_df = pd.read_csv(weather_string, index_col=0)
weather_df

,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,pevek,2122090,100,RU,1586547028,86,69.70,170.31,1.17,22.48
1,saint george,5546220,1,US,1586547016,44,37.10,-113.58,60.80,4.70
2,luderitz,3355672,0,NaN,1586547058,77,-26.65,15.16,60.80,4.70
3,chokurdakh,2126123,0,RU,1586547201,96,70.63,147.92,18.79,10.54
4,da nang,1583992,20,VN,1586547201,88,16.07,108.22,77.00,4.68
...,...,...,...,...,...,...,...,...,...,...
559,uarini,3661756,48,BR,1586547286,64,-2.99,-65.11,85.48,3.06
560,koygorodok,543396,100,RU,1586547286,93,60.45,51.00,31.51,8.12
561,beihai,1816705,98,CN,1586547286,90,21.48,109.10,73.33,12.03
562,zambezi,895953,96,ZM,1586547286,63,-13.54,23.10,65.73,5.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [171]:
# Configure gmaps
gmaps.configure(api_key= gkey)

In [172]:
locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"]
max_humid = weather_df["Humidity"].max
max_humid()

100

In [173]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [174]:
ideal_df = weather_df.loc[(weather_df["Max Temp"] > 80) & (weather_df["Humidity"] < 70) & 
                            (weather_df["Cloudiness"] < 30) & (weather_df["Wind Speed"] < 5)]
ideal_df

,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
112,bosaso,64013,0,SO,1586547216,64,11.28,49.18,82.71,3.89
341,birao,240210,8,CF,1586547208,14,10.28,22.79,85.98,4.81
374,urucara,3385670,29,BR,1586547257,69,-2.54,-57.76,86.90,1.54
439,thinadhoo,1337610,11,MV,1586547267,61,0.53,72.93,84.29,3.58
516,san juan,3837213,0,AR,1586547280,22,-31.54,-68.54,80.20,4.68


In [175]:
ideal_df.dropna()

,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
112,bosaso,64013,0,SO,1586547216,64,11.28,49.18,82.71,3.89
341,birao,240210,8,CF,1586547208,14,10.28,22.79,85.98,4.81
374,urucara,3385670,29,BR,1586547257,69,-2.54,-57.76,86.90,1.54
439,thinadhoo,1337610,11,MV,1586547267,61,0.53,72.93,84.29,3.58
516,san juan,3837213,0,AR,1586547280,22,-31.54,-68.54,80.20,4.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [176]:
import pandas as pd
import numpy as np
import requests
import json

In [177]:
hotel_df = ideal_df.dropna()

In [178]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
112,bosaso,64013,0,SO,1586547216,64,11.28,49.18,82.71,3.89,
341,birao,240210,8,CF,1586547208,14,10.28,22.79,85.98,4.81,
374,urucara,3385670,29,BR,1586547257,69,-2.54,-57.76,86.90,1.54,
439,thinadhoo,1337610,11,MV,1586547267,61,0.53,72.93,84.29,3.58,
516,san juan,3837213,0,AR,1586547280,22,-31.54,-68.54,80.20,4.68,


In [179]:
# create a params dict that will be updated with new city each iteration
params = {
    "radius" : 5000,
    "types": "lodging",
    "key": gkey}

In [180]:
# Loop through the cities_pd and run a lat/long search for each city

for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Long']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    results = requests.get(base_url, params=params).json()
    
    print(json.dumps(results, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = (results["results"][0]["name"])
    
    except:
        print("No hotel name.")

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAA_cMNE--1mL3KX96NaPR4tubQjFbvizpE7izuoAFDUz_2RtgKy_aLphdc01-4rBt5-Th8qIrSa7NYq_JTnOq-9Ke_mOkTWzdmKY0t6gmBCcE2WScylRQ9VVYRg2S-Vu2HRcIWnvPeTXI3WY9Vc26k1UjKLNtHwB7zss3RJewMRuR8JTl88ACeJ_VlE7Ihr5VjDqsOsHO0a-A9hr6PHxwyDW7W2qlC3kmUz9spaKoD_O3DvBIhwVoeRjlz3EB3EbLdnzxbXX1ZDTfx7BUpJFwfSeTFGq7m1VXhlrpPxGNzAxqEeiJLED_xvXsrtV4kFSsSmtoJn7lbavf1RmT9FKi0b3aZYgQ-Cu7CwdFIocgwChMwL8_csmCsbkYP6oa9dq5RIQKhFZRZAa9n08xOURwM-XPhoU9TDtOVcqF6mkIYBUcWBWtwP_hPU",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 11.2853253,
                    "lng": 49.17900069999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 11.2866437302915,
                        "lng": 49.18037983029149
                    },
                    "southwest": {
                        "lat": 11.2839457697085,
                        "lng": 49.17768

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -2.5375099,
                    "lng": -57.761021
                },
                "viewport": {
                    "northeast": {
                        "lat": -2.536202319708498,
                        "lng": -57.75967471970849
                    },
                    "southwest": {
                        "lat": -2.538900280291502,
                        "lng": -57.7623726802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "fbaa66c3fbf0880db7ea4b0cc15b4cbbfe159ff9",
            "name": "Pousada Maria Bonita",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 960,
                    "html_attributions": [
                  

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAOJ9EF0SEZRHS-ncwipG-kWai8az0rrEM5ROTFsFge5PjsmE4QpPP-c6ST6zKcNtXmoCEw3U6efO3k-1e7UJN37uWrlum3G0_kiIpbF2CKXaQmdbDsfoxsPWI9qW3QS4bWGCsH09_M1k_PFkH3XTeBxh2IY6WKLd51Y2hUHB98kgOcmI3Q6swEk4am57kKS5JE60BZpUsXjOn4VM_okG8ycNXqanDb8H-WyCApYtT2dXe7YSUiDTH4ooil3BwUlRmRX99vbMx0KpVscMb81DseIuO2cZtvKHhkJIeCN0-WlJ-Ijupraguelnu1-pwM4PFkKkXgk7q6Iph7X6edrpgm0tdU8MqsXbRayO85eaNWVncc17ZvSLCYTtCQXHSFuqmRIQ5uZWVzIQpGD99t7wcZ8hmRoUAQMuwfO7JVq6LO8uySaAErvdsRE",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.5370894,
                    "lng": -68.5239655
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.5357823197085,
                        "lng": -68.52261211970851
                    },
                    "southwest": {
                        "lat": -31.53848028029149,
                        "lng": -68.52531

In [181]:
hotel_df

,City,City ID,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
112,bosaso,64013,0,SO,1586547216,64,11.28,49.18,82.71,3.89,Red Sea Hotel
341,birao,240210,8,CF,1586547208,14,10.28,22.79,85.98,4.81,
374,urucara,3385670,29,BR,1586547257,69,-2.54,-57.76,86.90,1.54,Pousada Maria Bonita
439,thinadhoo,1337610,11,MV,1586547267,61,0.53,72.93,84.29,3.58,
516,san juan,3837213,0,AR,1586547280,22,-31.54,-68.54,80.20,4.68,Hotel Provincial


In [182]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [184]:
# Add marker layer ontop of heat map
# Display Map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))